In [ ]:
# Before executing all the code run the below commands in terminal 
# pip install pandas scipy

In [24]:
import os
from urllib.request import urlretrieve

def download_text(url):
    local_fname = os.path.basename(url)
    if not os.path.exists(local_fname):
        urlretrieve(url, local_fname)
    return local_fname

In [25]:
fname = download_text('https://www.gutenberg.org/files/2650/2650-0.txt')

In [26]:
def read_file(file):
    f = open(file, encoding='utf-8')
    l = f.read().splitlines()
    return list(filter(None, l))


x = read_file('2650-0.txt')

def index_of_substring(sub, l):
    return min([i for i, elem in enumerate(l) if sub in elem])

def read_data(x):
    start_index = index_of_substring('*** START', x)
    end_index = index_of_substring('*** END', x)
    return x[start_index+1:end_index]

data = read_data(x)
# print(read_data(x))

In [27]:
def count_vowel_and_consonants(d):
    vovel_count = 0
    consonant_count = 0
    for i in d:
        for j in i.lower():
            for k in j:
                if j in 'aeiouyàâæèéêëîïôùûüÿœ':
                    vovel_count += 1
                elif k.isalpha():
                    consonant_count += 1
    return vovel_count, consonant_count

print(count_vowel_and_consonants(data))

(360122, 426580)


In [28]:

def convert_diatrics_and_digraphs(data):
    d = {'à':'a', 'â':'a', 'è':'e', 'é':'e', 'ê':'e', 'ë':'e', 'î':'i', 'ï':'i', 'ô':'o', 'ù':'u', 'û':'u', 'ü':'u', 'ÿ':'y', 'æ':'ae', 'œ':'oe'}
    output = []

    for i in data:
        for key, value in d.items():
            i = i.replace(key, value)
            i = i.replace(key.upper(), value.upper())
        output.append(i)
    return output
answer = convert_diatrics_and_digraphs(data)
# print(convert_diatrics_and_digraphs(data))

In [29]:

def conversion_change(before_count, after_count):
    if after_count >= before_count:
        return '+' + str('{0:.2f}'.format(float(((after_count-before_count)/after_count)*100))) + '%'
    else:
        return '-' + str('{0:.2f}'.format(float(((before_count-after_count)/before_count)*100))) + '%'

def write_output_file(file):
    before_vowel_count, before_consonant_count = count_vowel_and_consonants(data)
    after_vowel_count, after_consonant_count = count_vowel_and_consonants(answer)
    with open(file, "r+") as f:
        f.write('From Project Gutenberg (www.gutenberg.org)')
        f.write('\n')
        f.write('Converted by: Ima Huskie')
        f.write('\n\n')
        f.write('*** START OF THE EBOOK ***')
        f.write('\n')
        f.writelines(answer)
        f.write('\n')
        f.write('*** END OF THE EBOOK ***')
        f.write('\n\n')
        f.write('Number of vowels before:' + '       ' + str(before_vowel_count))
        f.write('\n')
        f.write('Number of vowels after:' + '        ' + str(after_vowel_count))
        f.write('\n')
        f.write('Conversion Change:' + '             ' + conversion_change(before_vowel_count, after_vowel_count))
        f.write('\n\n')
        f.write('Number of consonants before:' + '   ' + str(before_consonant_count))
        f.write('\n')
        f.write('Number of consonants after:' + '    ' + str(after_consonant_count))
        f.write('\n')
        f.write('Conversion Change:' + '             ' + conversion_change(before_consonant_count, after_consonant_count))
        f.close()

write_output_file('2650-0-converted.txt')

In [30]:
fname2 = download_text('https://www.gutenberg.org/files/1342/1342-0.txt')

In [40]:
import pandas as pd
from scipy.stats import chi2_contingency

def compare_texts(file1, file2):
    filename1 = download_text(file1)
    filename2 = download_text(file2)
    f1 = read_file(filename1)
    f2 = read_file(filename2)
    data1 = read_data(f1)
    data2 = read_data(f2)
    data1 = convert_diatrics_and_digraphs(data1)
    data2 = convert_diatrics_and_digraphs(data2)
    vowel_count1, consonant_count1 = count_vowel_and_consonants(data1)
    vowel_count2, consonant_count2 = count_vowel_and_consonants(data2)
    indexes = [filename1, filename2, 'Total']
    dic = {'# of vowels': [vowel_count1, vowel_count2, vowel_count1 + vowel_count2], 
    '# of consonants': [consonant_count1, consonant_count2, consonant_count1 + consonant_count2], 
    'Total': [vowel_count1 + consonant_count1, vowel_count2 + consonant_count2, vowel_count1 + vowel_count2 + consonant_count1 + consonant_count2]}
    df = pd.DataFrame(dic, index=indexes)
    required_df = df.iloc[:2,:2]
    obs = required_df.to_numpy()
    chi2, p, dof, ex = chi2_contingency(obs)
    return df, 'p-value: ' + str(p)

In [41]:

compare_texts('https://www.gutenberg.org/files/2650/2650-0.txt', 'https://www.gutenberg.org/files/1342/1342-0.txt')

(            # of vowels  # of consonants    Total
 2650-0.txt       360385           426580   786965
 1342-0.txt       216806           320179   536985
 Total            577191           746759  1323950,
 'p-value: 0.0')

In [42]:

compare_texts('https://www.gutenberg.org/files/161/161-0.txt', 'https://www.gutenberg.org/files/1342/1342-0.txt')

(            # of vowels  # of consonants    Total
 161-0.txt        212215           313598   525813
 1342-0.txt       216806           320179   536985
 Total            429021           633777  1062798,
 'p-value: 0.8740327835853077')